In [ ]:
!pip install openai-agents

# Introduction to [OpenAI Agents SDK](https://openai.github.io/openai-agents-python/)

- Allows you to build agentic AI apps in a lightweight, easy-to-use package with very few abstractions. 
- Small set of primitives:
  - Agents: LLMs equipped with instructions and tools
  - Handoffs: allow agents to delegate to other agents for specific tasks
  - Guardrails: enable validation of agent inputs and outputs
  - Sessions: automatically maintains conversation history across agent runs

It also includes **built-in tracing**: lets you visualize and debug your agentic flows, as well as evaluate them and even fine-tune models for your application.

**2 design principles:**

1. Enough features to be worth using, but few enough primitives to make it quick to learn.
2. Works great out of the box, but you can customize exactly what happens.

## Main features:

- Agent loop: Built-in agent loop that handles calling tools, sending results to the LLM, and looping until the LLM is done.
- Python-first: Use built-in language features to orchestrate and chain agents, rather than needing to learn new abstractions.
- Handoffs: A powerful feature to coordinate and delegate between multiple agents.
- Guardrails: Run input validations and checks in parallel to your agents, breaking early if the checks fail.
- Sessions: Automatic conversation history management across agent runs, eliminating manual state handling.
- Function tools: Turn any Python function into a tool, with automatic schema generation and Pydantic-powered validation.
- Tracing: Built-in tracing that lets you visualize, debug and monitor your workflows, as well as use the OpenAI suite of evaluation, fine-tuning and distillation tools.

### Simple Code Example

In [3]:
# Set your openai API key
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"var: ")

_set_env("OPENAI_API_KEY")

In [4]:
# this import is for running it on a jupyter notebook
import nest_asyncio
nest_asyncio.apply()

from agents import Agent, Runner, function_tool

@function_tool
def read_file(file_path: str):
    """Reads the content of a text file and returns it as a string."""
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return f.read()
    except Exception as e:
        return f"Error reading file: {e}"

@function_tool
def write_file(file_path: str, content: str):
    """Writes the provided content into a text file."""
    try:
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(content)
        return "File written successfully."
    except Exception as e:
        return f"Error writing file: {e}"

# Setup your agent with custom instructions (here we're using the default model)
agent = Agent(
    name="FileAgent",
    instructions="You are a helpful assistant that can read from and write to files using read_file and write_file tools.",
    model="gpt-5-mini",
    tools=[read_file, write_file]
)

# Example usage: generate haiku and ask it to write to a file, then read back
prompt = (
    "Write a joke about an overly excited bald instructor, "
    "save it to a file called jokes_that_are_true.txt, "
)

result = Runner.run_sync(agent, prompt)

print(result.final_output)

Done — I saved the joke to jokes_that_are_true.txt. Here’s what I wrote:

Why did the overly excited bald instructor always get a standing ovation?
Because when he got enthusiastic his head shone so brightly the students thought the lesson had reached "enlightenment" — and nobody wanted to be the first to sit down.


# Let's Implement our First Little Research Agent! 

In [8]:
from agents import Agent, Runner, function_tool
from agents.tool import WebSearchTool



# i'm repeating these function definintions for the students
# that get distracted and forget I just did that
@function_tool
def read_file(file_path: str):
    """Reads the content of a text file and returns it as a string."""
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return f.read()
    except Exception as e:
        return f"Error reading file: {e}"

@function_tool
def write_file(file_path: str, content: str):
    """Writes the provided content into a text file."""
    try:
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(content)
        return "File written successfully."
    except Exception as e:
        return f"Error writing file: {e}"

agent = Agent(
    name="Jokes Research Agent",
    instructions="""
    You research online for either the current news as material for unique jokes
    or you create the jokes on the spot. If the request from the user is not about joke
    writing, you just say, 'I'm all about jokes bro'.
    """,
    model='gpt-5-mini',
    tools=[write_file, read_file,WebSearchTool()]
)

prompt = """
Read up news about the latest developments in AI,
and write 3 jokes about those topics.
Save the jokes with the referenced sources into a file
called research_jokes.txt
"""

result = Runner.run_sync(agent, prompt)

result.final_output

"Done — I read recent AI coverage and saved three topical jokes (with sources) into research_jokes.txt.\n\nWhat I read (key items):\n- Disney agreed to a major investment enabling character use in OpenAI's Sora video tool (The Guardian). \ue200cite\ue202turn1view0\ue201  \n- OpenAI warned that more-capable models could pose a high cybersecurity risk. \ue200cite\ue202turn1view1\ue201  \n- Google is updating AI Mode to show more source links. \ue200cite\ue202turn2view0\ue201  \n- DeepMind is building an automated science laboratory in the UK and has reported major problem‑solving advances. \ue200cite\ue202turn4view0\ue202turn3view2\ue201\n\nThe 3 jokes (also saved to research_jokes.txt):\n\n1) Disney + OpenAI\nDisney just gave OpenAI big money to put characters into Sora — finally a legally licensed way for your chatbot to say “Oh boy!” and mean it. \ue200cite\ue202turn1view0\ue201\n\n2) Cybersecurity warning\nOpenAI warned newer models could invent zero‑day exploits — great, now the AI 

In [9]:

prompt = "Read the jokes from research_jokes.txt"
result = Runner.run_sync(agent, prompt)

In [10]:
result.final_output

'Got it — here are the jokes from research_jokes.txt:\n\nGenerated on: December 11, 2025\n\n1) Joke:\nDisney just gave OpenAI $1 billion to put Mickey in Sora videos — finally a legally licensed way for your chatbot to say "Oh boy!" without getting sued. (But please: no celebrity voice impersonations.)\nSources:\n- "Disney to invest $1bn in OpenAI, allowing characters in Sora video tool" — The Guardian — December 11, 2025. \ue200cite\ue202turn1view0\ue201\n\n2) Joke:\nOpenAI warned its next models could invent zero-day exploits — great, now the AI won\'t just debug your code, it\'ll debug your company out of existence. \nSources:\n- "OpenAI warns new models pose \'high\' cybersecurity risk" — Reuters — December 10, 2025. \ue200cite\ue202turn1view1\ue201\n\n3) Joke:\nGoogle says AI Mode will link more sources so you can trace an AI\'s claim — progress! Meanwhile DeepMind\'s robots are building materials labs so fast, your replacement coworker will arrive stamped and assembled. \nSources

## Jokes from `research_jokes.txt`

**Generated on:** December 11, 2025

---

### 1) Joke

Disney just gave OpenAI $1 billion to put Mickey in Sora videos — finally a legally licensed way for your chatbot to say "Oh boy!" without getting sued.  
*(But please: no celebrity voice impersonations.)*

**Sources:**
- "Disney to invest $1bn in OpenAI, allowing characters in Sora video tool" — The Guardian — December 11, 2025.

---

### 2) Joke

OpenAI warned its next models could invent zero-day exploits — great, now the AI won't just debug your code, it'll debug your company out of existence.

**Sources:**
- "OpenAI warns new models pose 'high' cybersecurity risk" — Reuters — December 10, 2025.

---

### 3) Joke

Google says AI Mode will link more sources so you can trace an AI's claim — progress!  
Meanwhile DeepMind's robots are building materials labs so fast, your replacement coworker will arrive stamped and assembled.

**Sources:**
- "Google says it will link to more sources in AI Mode" — The Verge — December 10, 2025.
- "Google DeepMind to build its first 'automated science laboratory' in the UK" — The Guardian — December 11, 2025.
- "Google DeepMind claims 'historic' AI breakthrough in problem solving" — The Guardian — September 17, 2025.

---

*(End of file)*